# Extracting Outlines from PDFs

The objective of this piece of code is to go through all the PDFs and detect if there are outlines present. The output of this code is the following: <br>

1. Outline_Present.csv - This has the list of the PDF files and then there is an indicator is the outline i spresent or not. <br>

2. Index 4 - Chapters.csv - This table has a detailed list of all the chapters present in all the PDF files present. 

In [75]:
import PyPDF2 as p2
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
import pandas as pd
import os
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [76]:
#define the paths of the PDF scrapped 

path = 'F:/Environmental Baseline Data/Version 3/Data/PDF'
pdfs = os.listdir(path)
pdf_paths = [str(path) + '/' + str(x) for x in pdfs]
len(pdf_paths)

1329

## Extract Outlines for all the PDFs

In [77]:
# Functtion that runs recursively to extract the outlines of the PDF files
def nested_is_instance_check(s, level, file_name, file_names_ch, chapters, levels, pages):
    level +=1
    if isinstance(s, (int, list, float, complex)):
        for x in s:
            nested_is_instance_check(x, level, file_name, file_names_ch, chapters, levels, pages)
    else:
        chapters.extend([s.title])
        levels.extend([level])
        #it was observed that for a lot of PDF files we could not find the page number 
        # Hence, -999 is the error code added for those cases 
        try:
            pages.extend([pdfread.getDestinationPageNumber(s)])
        except: 
            pages.extend([-999])
        file_names_ch.append([file_name])

In [110]:
# Varibles for file outline_present.csv
file_names = []
outline_present = []

# Variables for Index 4 - Chapters.csv
file_names_ch = []
chapters = []
levels = []
pages = []


#pdf_paths = pdf_paths[0:10]

for pdf_path in pdf_paths:
    file_name = str(pdf_path.split('/')[-1])
    file_names.append(file_name)
    try:
        pdfread = p2.PdfFileReader(pdf_path)
        s = pdfread.outlines
        len_s = len(s)
    except:
        outline_present.append("Error in reading the outline")
        
    if len_s > 0:
        level = 0
        outline_present.append(1)
        nested_is_instance_check(s, level, file_name, file_names_ch, chapters, levels, pages)
    elif len_s == 0:
        outline_present.append(0)
    
    # Error Code for s (=pdfraed.outlines) is -999
    # the negative number allows exclusivity from the if and the elif conditions above
    len_s = -999 

#### Saving the file outline_present.csv

In [112]:
len(file_names)
len(pdf_paths)
len(outline_present)

1329

1329

1329

In [80]:
df_outlines = pd.DataFrame({'file_names': file_names, 'pdf_paths' : pdf_paths, 'outline_present' : outline_present})
df_outlines.head(5)
df_outlines.to_csv("F:/Environmental Baseline Data/Version 3/Indices/Outline_Present.csv")

,file_names,pdf_paths,outline_present
0,A0H8C0.pdf,F:/Environmental Baseline Data/Version 3/Data/...,1
1,A0H8C1.pdf,F:/Environmental Baseline Data/Version 3/Data/...,1
2,A0H8C2.pdf,F:/Environmental Baseline Data/Version 3/Data/...,0
3,A0H8C3.pdf,F:/Environmental Baseline Data/Version 3/Data/...,0
4,A0U3G1.pdf,F:/Environmental Baseline Data/Version 3/Data/...,0


#### Saving the file Index 4 - Chapters.csv

In [81]:
len(file_names_ch)
len(chapters)
len(pages)
len(levels)

19439

19439

19439

19439

In [109]:
df_chapters = pd.DataFrame({'file_names': file_names_ch, 'chapters': chapters, 'pages' : pages, 'levels' : levels})
df_chapters.head(5)

df_chapters.to_csv("F:/Environmental Baseline Data/Version 3/Indices/Index 4 (Outlines) - Chapters and everything else.csv")


##########################################################################################
########## We might need to think of a better way to extract important chapters ##########
##########  df.chapters.levels < 5, is not teh best way ##################################
##########################################################################################
df_chapters = df_chapters[df_chapters.chapters is not None  and df_chapters.levels < 5]
df_chapters.head(5)
df_chapters.to_csv("F:/Environmental Baseline Data/Version 3/Indices/Index 4 (Outlines)- Chapters.csv")



,file_names,chapters,pages,levels
0,[A0H8C0.pdf],Environmental Impact Assessment and Mitigation...,24,2
1,[A0H8C0.pdf],Introduction,24,3
2,[A0H8C0.pdf],Project Background,24,4
3,[A0H8C0.pdf],Regulatory Setting,24,4
4,[A0H8C0.pdf],,-1,5


,file_names,chapters,pages,levels
0,[A0H8C0.pdf],Environmental Impact Assessment and Mitigation...,24,2
1,[A0H8C0.pdf],Introduction,24,3
2,[A0H8C0.pdf],Project Background,24,4
3,[A0H8C0.pdf],Regulatory Setting,24,4
9,[A0H8C0.pdf],Scope of Environmental Assessment,26,4


PermissionError: [Errno 13] Permission denied: 'F:/Environmental Baseline Data/Version 3/Indices/Index 4 (Outlines)- Chapters.csv'

## Is Table of contents Present?

In [106]:
txt_path = "F:/Environmental Baseline Data/Version 3/Data/Text/"
TOC_present = []
txt_file_needed = []
for file_name in file_names:
    # additional check to get rid of the string "Thumb"
    if 'Thum' not in file_name:  
        file_name_no_ext = file_name[:-4]
        
        try:
            file_handle = open(txt_path + file_name_no_ext +'.txt',"r", encoding="utf8")
            if file_handle.mode == 'r':
                contents = file_handle.read()
                contents = contents.upper()
                if 'TABLE OF CONTENTS' in contents:
                    TOC_present.append('TOC Present')
                else:
                    TOC_present.append('TOC Not Present')
            else:
                TOC_present.append('Error')
                
        except:
            print("This file was not present as text {}".format(file_name_no_ext))
            txt_file_needed.append(file_name_no_ext)
            TOC_present.append("Text file not present ")
            
    else:
        TOC_present.append("Thumb File")
    

This file was not present as text A5A8L7
This file was not present as text A5A8L8
This file was not present as text A5A8L9
This file was not present as text A5A8Q0
This file was not present as text A5A8Q1
This file was not present as text A5A8Q2
This file was not present as text A5A8Q3
This file was not present as text A5A8Q4
This file was not present as text A5A8Q5
This file was not present as text A5A8Q6
This file was not present as text A5A8Q7
This file was not present as text A5A8R1
This file was not present as text A5A8R2
This file was not present as text A5A8R3
This file was not present as text A5A8R4
This file was not present as text A5A8R5
This file was not present as text A5A8R6
This file was not present as text A5A8R7
This file was not present as text A5A8R8
This file was not present as text A5A8R9
This file was not present as text A5A8S0
This file was not present as text A5A8S1
This file was not present as text A5A8S2
This file was not present as text A5A8S3
This file was no

This file was not present as text A5V8G2
This file was not present as text A6A2C8
This file was not present as text A6A2C9
This file was not present as text A6A2D0
This file was not present as text A6A2D1
This file was not present as text A6A2D2
This file was not present as text A6A2D3
This file was not present as text A6A2D4
This file was not present as text A6A2D5
This file was not present as text A6A2D6
This file was not present as text A6C8K0
This file was not present as text A6F4Q3
This file was not present as text A6F4Q4
This file was not present as text A6F4Q5
This file was not present as text A6F4Q6
This file was not present as text A6F4Q7
This file was not present as text A6F4Q8
This file was not present as text A6F4Q9
This file was not present as text A6F4R0
This file was not present as text A6F4R1
This file was not present as text A6F4R2
This file was not present as text A6I8E1
This file was not present as text A6T2G2
This file was not present as text A6T2G3
This file was no

In [89]:
len(txt_file_needed)

262

In [90]:
len(TOC_present)
len(file_names)

1329

1329

In [91]:
df_outlines['TOC_present'] = TOC_present
df_outlines.head(5)
df_outlines.to_csv("F:/Environmental Baseline Data/Version 3/Indices/Outline_Present.csv")

,file_names,pdf_paths,outline_present,TOC_present
0,A0H8C0.pdf,F:/Environmental Baseline Data/Version 3/Data/...,1,TOC Present
1,A0H8C1.pdf,F:/Environmental Baseline Data/Version 3/Data/...,1,TOC Not Present
2,A0H8C2.pdf,F:/Environmental Baseline Data/Version 3/Data/...,0,TOC Not Present
3,A0H8C3.pdf,F:/Environmental Baseline Data/Version 3/Data/...,0,TOC Not Present
4,A0U3G1.pdf,F:/Environmental Baseline Data/Version 3/Data/...,0,TOC Present


# Postal to Int Functions 

In [92]:
def postal_to_int(Postal):
    multiplier = 1
    odd = 1
    Postal_Int_Word = 0
    Error_Message = "Invalid Postal Code"
    
    # Reversing the string 
    Postal = Postal[::-1] 
    
    # Chcek for the length of the Postal Code
    if not(len(Postal)==6):
        return(Error_Message)
    
    for letter in list(str.strip(Postal)):
        if odd%2 == 1:
            # Check if it is an interger or not   
            # If this is not an integer then exit
            if not (str.isnumeric(letter)):
                return(Error_Message)
            
            Postal_Int_Letter = multiplier * int(letter)
            Postal_Int_Word = Postal_Int_Word + Postal_Int_Letter
            
            odd = odd + 1
            multiplier = multiplier * 10
            # print(letter, Postal_Int_Letter)
            # print(Postal_Int_Word)
        else:
            # Check if it is a character or not
            # If this is not a character then exit
            if not (str.isalpha(letter)):
                return(Error_Message)
            
            # ord('A') = 65
            Postal_Int_Letter = multiplier * (ord(letter) - 65)
            Postal_Int_Word = Postal_Int_Word + Postal_Int_Letter
            
            odd = odd + 1
            multiplier = multiplier * 26
            # print(letter, Postal_Int_Letter)
            # print(Postal_Int_Word)
            
    return Postal_Int_Word

In [93]:
# never used integer to Postal but I guess it would be useful 
# Also useful in validating if Postal to Int Function works 

def int_to_postal(Postal_Int_Word):
    Postal_Int_Word_Rem = Postal_Int_Word
    Postal = ""
    odd = 0
    Error_Message = "Invalid Postal Code"
    
    # Reversing the string 
    Postal = Postal[::-1] 
 
    while odd < 6:
        if odd%2 == 0:
            Postal_Int_Letter = Postal_Int_Word_Rem % 10
            Postal = Postal + str(Postal_Int_Letter)
            Postal_Int_Word_Rem = int(Postal_Int_Word_Rem/10)
            odd = odd + 1

        else:
            Postal_Int_Letter = (Postal_Int_Word_Rem % 26) +65
            Postal = Postal + chr(Postal_Int_Letter)
            Postal_Int_Word_Rem = int(Postal_Int_Word_Rem/26)
            odd = odd + 1
            
    # Accounts for 7 digit Postal Codes
    if Postal_Int_Word_Rem > 0:
        return(Error_Message)
        
    return Postal[::-1]
  

In [94]:
int_postals = []
for file_name in file_names:
    postal = file_name[:-4]
    int_postal = postal_to_int(postal)
    int_postals.append(int_postal)
    
len(int_postals)
len(file_names)

1329

1329

In [95]:
df_outlines['Int_Postal_Codes'] = int_postals
df_outlines.head(5)
df_outlines.to_csv("F:/Environmental Baseline Data/Version 3/Indices/Outline_Present.csv")

,file_names,pdf_paths,outline_present,TOC_present,Int_Postal_Codes
0,A0H8C0.pdf,F:/Environmental Baseline Data/Version 3/Data/...,1,TOC Present,20300
1,A0H8C1.pdf,F:/Environmental Baseline Data/Version 3/Data/...,1,TOC Not Present,20301
2,A0H8C2.pdf,F:/Environmental Baseline Data/Version 3/Data/...,0,TOC Not Present,20302
3,A0H8C3.pdf,F:/Environmental Baseline Data/Version 3/Data/...,0,TOC Not Present,20303
4,A0U3G1.pdf,F:/Environmental Baseline Data/Version 3/Data/...,0,TOC Present,52841


In [96]:
chunk_nos = []
chunk = 0
last_postal = int_postals[0]

for int_postal in int_postals:
    if int_postal == last_postal + 1:
        chunk_nos.append(chunk)
    else:
        chunk = chunk +1
        chunk_nos.append(chunk)
    last_postal = int_postal

In [97]:
df_outlines['chunk_nos'] = chunk_nos
df_outlines.head(5)
df_outlines.to_csv("F:/Environmental Baseline Data/Version 3/Indices/Outline_Present.csv")

,file_names,pdf_paths,outline_present,TOC_present,Int_Postal_Codes,chunk_nos
0,A0H8C0.pdf,F:/Environmental Baseline Data/Version 3/Data/...,1,TOC Present,20300,1
1,A0H8C1.pdf,F:/Environmental Baseline Data/Version 3/Data/...,1,TOC Not Present,20301,1
2,A0H8C2.pdf,F:/Environmental Baseline Data/Version 3/Data/...,0,TOC Not Present,20302,1
3,A0H8C3.pdf,F:/Environmental Baseline Data/Version 3/Data/...,0,TOC Not Present,20303,1
4,A0U3G1.pdf,F:/Environmental Baseline Data/Version 3/Data/...,0,TOC Present,52841,2


In [98]:
for current_chunk in range(0,chunk):
    current_df = df_outlines[df_outlines.chunk_nos == current_chunk]
    if current_chunk > 2:
        break
        
current_df

,file_names,pdf_paths,outline_present,TOC_present,Int_Postal_Codes,chunk_nos
6,A0Y1T7.pdf,F:/Environmental Baseline Data/Version 3/Data/...,1,TOC Present,62857,3
7,A0Y1T8.pdf,F:/Environmental Baseline Data/Version 3/Data/...,1,TOC Not Present,62858,3
8,A0Y1T9.pdf,F:/Environmental Baseline Data/Version 3/Data/...,1,TOC Not Present,62859,3
9,A0Y1U0.pdf,F:/Environmental Baseline Data/Version 3/Data/...,1,TOC Not Present,62860,3
10,A0Y1U1.pdf,F:/Environmental Baseline Data/Version 3/Data/...,Error in reading the outline,TOC Not Present,62861,3
11,A0Y1U2.pdf,F:/Environmental Baseline Data/Version 3/Data/...,Error in reading the outline,TOC Not Present,62862,3
12,A0Y1U3.pdf,F:/Environmental Baseline Data/Version 3/Data/...,Error in reading the outline,TOC Not Present,62863,3
13,A0Y1U4.pdf,F:/Environmental Baseline Data/Version 3/Data/...,Error in reading the outline,TOC Not Present,62864,3
14,A0Y1U5.pdf,F:/Environmental Baseline Data/Version 3/Data/...,0,TOC Not Present,62865,3
15,A0Y1U6.pdf,F:/Environmental Baseline Data/Version 3/Data/...,0,TOC Not Present,62866,3
